<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial: compute_conservs_and_Tmunu.c

## Authors: Leo Werneck, Zach Etienne, and Samuel Cupp

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the function `compute_conservs_and_Tmunu`. This function uses primitives to compute the conservative quantities and stress energy tensor $T_{\mu\nu}$.

### Required and recommended citations:
* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#introduction): **Introduction**
1. [Step 2](#compute_conservs_and_Tmunu): **The `compute_conservs_and_Tmunu` function**

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

In this tutorial notebook we discuss our code for computing the conservatives and stress-energy tensor $T_{\mu\nu}$. This is usually used after the conservative-to-primitive routine and `enforce_primitive_limits_and_output_u0` have run to ensure these quantities are consistent with the newly computed primitives.

<a id='compute_conservs_and_Tmunu'></a>

# Step 2: The `compute_conservs_and_Tmunu` function \[Back to [top](#toc)\]

$$\label{compute_conservs_and_Tmunu}$$


First, let's look at the arguments of `compute_conservs_and_Tmunu`:
```
void compute_conservs_and_Tmunu(
      const GRHayL_parameters *restrict params,
      const eos_parameters *restrict eos,
      const metric_quantities *restrict metric,
      const primitive_quantities *restrict prims,
      conservative_quantities *restrict cons,
      stress_energy *restrict Tmunu)
```
The first two arguments appear in most GRHayL functions and contain information on the simulation and equation-of-state parameters. The next struct contains (pointwise) information on the metric. Whenever packing the metric struct, we recommend using the function `GRHayL_enforce_detgtij_and_initialize_metric`, which also enforces that the determinant of the conformal metric equals one. GRHayL functions may behave unpredictably with abnormal metrics, as they have not been tested as rigorously outside the expected input ranges. The final inputs are the primitives. **Please note that this function requires that prims->u0 is set.**

The outputs are the conservatives and $T_{\mu\nu}$.

To compute the conservatives, we first compute a handful of intermediate quantities. We compute the enthalpy from epsilon, the pressure, and baryonic density. We have $u^0$ as an input, and we compute the remainder of the 4-velocity using
$$u^i = u^0 v^i$$
We use the helper function `lower_vector` to find $u_\mu$. We then compute
$$b^\mu\equiv \frac{B^i_u}{\sqrt{4 \pi}}$$
(Eq. 31 of http://arxiv.org/pdf/astro-ph/0503420.pdf) and $b^2$ using the function `compute_smallb_and_b2`. The time component is computed using Eq. 23 in http://arxiv.org/pdf/astro-ph/0503420.pdf
$$b^t = \frac{B^i_u}{\sqrt{4 \pi}} = \frac{u_i B^i}{\alpha\sqrt{4\pi}}$$
and spatial components $b^i$ are computed using Eq. 24 in http://arxiv.org/pdf/astro-ph/0503420.pdf
\begin{align}
b^i &= \frac{B^i_u}{\sqrt{4\pi}} \\
&= \frac{1}{u^0 \sqrt{4\pi}}\left(\frac{B^i}{\alpha} + B^0_u u^i\right) \\
&= \frac{1}{u^0 \sqrt{4\pi}}\left(\frac{B^i}{\alpha} + \sqrt{4\pi} b^t u^i\right) \\
&= \frac{1}{u^0 \sqrt{4\pi}}\left(B^i + \alpha\sqrt{4\pi} b^t u^i\right) \\
&= \frac{1}{u^0 \sqrt{4\pi}}\left(\frac{B^i}{u^0} +  \alpha\sqrt{4\pi} b^t \frac{u^i}{u^0}\right) \\
&= \frac{1}{u^0 \sqrt{4\pi}}\left(\frac{B^i}{u^0} +  \alpha\sqrt{4\pi} b^t v^i\right)
\end{align}
We then compute $b^2$
\begin{align}
b^2 &= g_{\mu \nu} b^{\mu} b^{\nu} \\
&= g_{tt} \left(b^t\right)^2 + g_{xx} \left(b^x\right)^2 + g_{yy} \left(b^y\right)^2 + g_{zz} \left(b^z\right)^2 + 2 (g_{tx} b^t b^x + g_{ty} b^t b^y + g_{tz} b^t b^z + g_{xy} b^x b^y + g_{xz} b^x b^z + g_{yz} b^y b^z) \\
&= (-\alpha^2 + \gamma_{ij} \beta^i \beta^j) \left(b^t\right)^2 + b^i b^j \gamma_{ij} + 2 g_{t i} b^t b^i \\
&= - (\alpha b^t)^2 + (b^t)^2 \gamma_{ij} \beta^i \beta^j + b^i b^j \gamma_{ij} + 2 b^t g_{t i} b^i \\
&= - (\alpha b^t)^2 + (b^t)^2 \gamma_{ij} \beta^i \beta^j + b^i b^j \gamma_{ij} + 2 b^t (\gamma_{ij} \beta^j) b^i \\
&= - (\alpha b^t)^2 + \gamma_{ij} \left((b^t)^2 \beta^i \beta^j + b^i b^j + 2 b^t \beta^j b^i\right) \\
&= - (\alpha b^t)^2 + \gamma_{ij} \left((b^t)^2 \beta^i \beta^j + 2 b^t \beta^j b^i + b^i b^j\right) \\
&= - (\alpha b^t)^2 + \gamma_{ij} \left(b^i + b^t \beta^i) (b^j + b^t \beta^j\right)
\end{align}

We also compute $b_\mu$, again using `lower_vector`. We after precomputing some intermediate quantities, we're ready to compute the conservatives. These are given by the equations
$$\tilde{\rho} = \alpha\sqrt{\gamma} \rho_b u^0$$
$$\tilde{S}_i = h \tilde{\rho} u_i + \alpha\sqrt{\gamma} \left( u^0 b^2 u_i - b^0 b_i \right)$$
$$\tilde{\tau} = \alpha^2\sqrt{\gamma}\left( (\rho_b h + b^2)\left(u^0\right)^2 - \frac{\left(P+\frac{b^2}{2}\right)}{\alpha^2} - \left(b^0\right)^2 - \tilde{\rho} \right)$$
where $h$ is the enthalpy.

Reusing many intermediate quantities, the stress-energy tensor is given by
$$T^{\mu\nu} = (\rho_b h + b^2)u_\mu u_\nu + \left(P+\frac{b^2}{2}\right)g_{\mu\nu} -b_\mu b_\nu$$